<a href="https://colab.research.google.com/github/bidapnagraj77/miniproject/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bidapnagraj77/test.git

Cloning into 'test'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 87 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (87/87), done.


In [2]:
!git clone https://github.com/bidapnagraj77/validation.git

Cloning into 'validation'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 65 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (65/65), done.


In [3]:
import numpy as np 
import os 
import cv2 
import random 
import threading


def create_data(datadir,save_filename_images,save_filename_labels,categories,image_size):
    '''
    datadir = give the directory of the folder containing images of different class\n
    destination = give the destination directory\n
    categories = a list containing the name of subdirectory\n
    image_size = size of image\n    
    label = "train" or "test"\n
    \n
    Example:\n
    Let the directory structure be\n
    IMAGE DATA -\n
        |\n
        |---CLASS_1\n
                |\n
                |-- 1.jpg\n
                |-- 2.jpg\n
        |\n
        |---CLASS_2\n
                |-- 1.jpg\n
                |-- 2.jpg\n
        |\n
        |---CLASS_3\n
                |-- 1.jpg\n
                |-- 2.jpg\n
    
    Then\n 
    datadir = "IMAGE DATA"\n
    categories = ["CLASS_1","CLASS_2","CLASS_2"]\n
    image_size = (224,224)\n    
    label = "train"
    '''
    data = []
    for category in categories:
        print("Current Directory:",datadir)
        print("Current class:",category)
        path = os.path.join(datadir,category)
        class_num = categories.index(category)
        files = os.listdir(path)
        total = len(files)
        print("Total:",total)
        current = 1
        for img in files: 
            print("Getting:", current, "of", total)              
            try:
                img_array = cv2.imread(os.path.join(path,img))
                img_array = cv2.resize(img_array,(image_size,image_size))
                data.append([img_array,class_num])                
            except Exception as e:
                pass 
            current += 1        
        print()          
    random.shuffle(data) 
    images = []
    classes = []    
    current = 1
    for image, cls in data:      
      images.append(image)
      classes.append(cls)
      current += 1
    
    print()  
    images = np.array(images).reshape(-1,image_size,image_size,3)
    images = images/255.0
    classes = np.array(classes)    
    print("Preparing .npy files....")
    np.save(save_filename_images,images)
    np.save(save_filename_labels,classes)
    print("npy Files Ready")

TRAIN_DIR = input("Enter the path to the training images: ")
TEST_DIR = input("Enter the path to the testing images: ")
DESTINATION = input("Enter the path to save the npy files: ")
CLASSES = ["","COVID_19 -ve"]
IMAGE_SIZE = 224

train_image_path = DESTINATION + "/New_Btrain_images.npy"   
train_labels_path = DESTINATION + "/New_Btrain_labels.npy"   
test_image_path = DESTINATION + "/New_Btest_images.npy"
test_labels_path = DESTINATION + "/New_Btest_labels.npy"

b_tr_thread = threading.Thread(target=create_data,args=(TRAIN_DIR,train_image_path,train_labels_path,CLASSES,IMAGE_SIZE))
b_te_thread = threading.Thread(target=create_data,args=(TEST_DIR,test_image_path,test_labels_path,CLASSES,IMAGE_SIZE))

b_tr_thread.start()
b_te_thread.start()
b_tr_thread.join()
b_te_thread.join()

Enter the path to the training images: /content/test/Train
Enter the path to the testing images: /content/validation/Val
Enter the path to save the npy files: /content/test
Current Directory:Current Directory: /content/validation/Val
Current class: 
 Total: 2
Getting: 1 of 2
/content/test/Train
Current class: 
Total: 2
Getting: 1 of 2
Getting:Getting: 2 of 2
 2 of 2

Current Directory: /content/validation/Val
Current class: COVID_19 -ve
Current Directory: /content/test/Train
Current class: COVID_19 -ve

Total: 45
Getting: 1 of 45
Total: 30
Getting: 1 of 30
Getting: 2 of 45
Getting: 2 of 30
Getting: 3 of 30
Getting: 3 of 45
Getting: 4 of 30
Getting: 4 of 45
Getting: 5 of 30
Getting: 5 of 45
Getting: 6 of 30
Getting: 6 of 45
Getting: 7 of 30
Getting: 7 of 45
Getting: 8 of 30
Getting: 8 of 45
Getting: 9 of 30
Getting: 10 of 30
Getting: 9 of 45
Getting: 11 of 30
Getting: 10 of 45
Getting: 12 of 30
Getting: 13 of 30
Getting: 11 of 45
Getting: 14 of 30
Getting: 12 of 45
Getting: 15 of 30
Get

In [7]:
%%writefile Model.py
import tensorflow as tf 
from tensorflow.keras.applications import VGG19, ResNet50V2, DenseNet201, InceptionResNetV2, InceptionV3, Xception, MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

def train_model(path, train_images=None, train_labels = None, 
                test_images = None, test_labels = None, 
                model_name = None, epochs =80, learning_rate = 0.0001,
                input_shape = (224,224,3), classes=2, batch_size = 16, 
                classifier_activation='softmax',
                callbacks = None):
    '''    
    saves the model as .h5 file\n  
    path = directory for saving the files
    train_images = a numpy array containing the image data for training\n
    train_labels = a numpy array containing the labels for training\n
    test_images = a numpy array containing the image data for test\n
    test_labels = a numpy array containing the labels for test\n
    model_name = a string, name of the model -> "vgg19", "resnet50_v2", "inception_resnet_v2", "densenet201", "inception_v3", "xception", "mobilenet_v2"\n
    epochs\n
    learning_rate\n        
    '''

    base_model = None
    if model_name == 'vgg19':        
        base_model = VGG19(weights = None, include_top = False, input_shape = input_shape)
     
    if model_name == 'resnet50_v2':
        base_model = ResNet50V2(weights = None, include_top = False, input_shape = input_shape)
             
    if model_name == 'inception_resnet_v2':        
        base_model = InceptionResNetV2(weights = None, include_top = False, input_shape = input_shape)
    
    if model_name == 'densenet201':        
        base_model = DenseNet201(weights = None, include_top = False, input_shape = input_shape)
          
    if model_name == 'inception_v3':        
        base_model = InceptionV3(weights = None, include_top = False, input_shape = input_shape)
        
    if model_name == 'xception':        
        base_model = Xception(weights = None, include_top = False, input_shape = input_shape)
        
    if model_name == 'mobilenet_v2':        
        base_model = MobileNetV2(weights = None, include_top = False, input_shape = input_shape)

    x = base_model.output         
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    output = tf.keras.layers.Dense(classes, activation=classifier_activation)(x)

    model = tf.keras.Model(inputs = base_model.input, outputs = output)

    optimizer = Adam(learning_rate = learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

    model.compile(optimizer = optimizer,
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics = ['accuracy'])
        
    results = model.fit(train_images, train_labels, epochs = epochs,
                        validation_data = (test_images, test_labels), 
                        batch_size=batch_size, 
                        callbacks = callbacks
                        )
    
    #losses = pd.DataFrame(model.history.history)
    #losses[['loss','val_loss']].plot()
    
    save_model = path + model_name + '.h5'
    model.save(save_model)
    
    return results    

Writing Model.py


In [8]:
import tensorflow as tf
import numpy as np
from Model import *


classes = ["COVID_19 +ve","COVID_19 -ve"]
image_size = 224

x_train_path = input("Enter path to train images: ")
y_train_path = input("Enter path to train labels: ")
x_test_path = input("Enter path to test images: ")
y_test_path = input("Enter path to test labels: ")
path = input("Enter path to save the model: ")  #path to save the model


x_train = np.load(x_train_path)
y_train = np.load(y_train_path)
x_test = np.load(x_test_path)
y_test = np.load(y_test_path)

inception = train_model(path, x_train, y_train,
                        x_test, y_test, model_name = "inception_v3",
                        epochs = 60, input_shape = (image_size,image_size,3),
                        classes = len(classes))

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, 
                           mode='min', restore_best_weights=True)
callbacks = [early_stop]

densenet = train_model(path, x_train, y_train,
                       x_test, y_test, model_name="densenet201",
                       epochs=60, input_shape = (image_size,image_size,3),
                       classes = len(classes),
                       callbacks = callbacks)

resnet = train_model(path, x_train, y_train,
                     x_test, y_test, model_name="resnet50_v2",
                     epochs=60, input_shape = (image_size,image_size,3),
                     classes = len(classes),
                     callbacks = callbacks)

Enter path to train images: /content/test/New_Btrain_images.npy
Enter path to train labels: /content/test/New_Btrain_labels.npy
Enter path to test images: /content/test/New_Btest_images.npy
Enter path to test labels: /content/test/New_Btest_labels.npy
Enter path to save the model: /content/validation
Epoch 1/60


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


3/3 [==============================] - 45s 3s/step - loss: 0.7706 - accuracy: 0.3778 - val_loss: 0.6913 - val_accuracy: 1.0000
Epoch 2/60
3/3 [==============================] - 1s 384ms/step - loss: 0.4489 - accuracy: 0.9778 - val_loss: 0.6797 - val_accuracy: 1.0000
Epoch 3/60
3/3 [==============================] - 1s 388ms/step - loss: 0.2219 - accuracy: 1.0000 - val_loss: 0.6527 - val_accuracy: 1.0000
Epoch 4/60
3/3 [==============================] - 1s 393ms/step - loss: 0.1168 - accuracy: 1.0000 - val_loss: 0.6262 - val_accuracy: 1.0000
Epoch 5/60
3/3 [==============================] - 1s 386ms/step - loss: 0.0610 - accuracy: 1.0000 - val_loss: 0.5979 - val_accuracy: 1.0000
Epoch 6/60
3/3 [==============================] - 1s 387ms/step - loss: 0.0373 - accuracy: 1.0000 - val_loss: 0.5694 - val_accuracy: 1.0000
Epoch 7/60
3/3 [==============================] - 1s 391ms/step - loss: 0.0234 - accuracy: 1.0000 - val_loss: 0.5452 - val_accuracy: 1.0000
Epoch 8/60
3/3 [=================

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [9]:
%%writefile Ensembling.py
import numpy as np 
import tensorflow as tf 

image_size = 224
classes = ["COVID_19 +ve","COVID_19 -ve"]

def ensemble(x, weights, models): 
    '''
    returns a weighted average of predictions made by the models\n
    x -> input image \n
    weights -> a list of weights \n
    models -> a list of models\n    
    '''      
    outputs = []    
    for model in models:                
        outputs.append(list(model.predict(x)[0]))                
    
    outputs = np.array(outputs)
    avg = np.average(a=outputs,axis=0,weights=weights)
    return avg


def equal(pred, label):
  pred_id = np.argmax(pred)
  if (pred_id == label):
    return True
  else:
    return False


def accuracy(predicted_values, y_truths):
  '''
  returns accuracy\n
  predicted_values = a numpy array containing the predictions\n
  y_truths = a numpy array containing the truth values\n  
  '''
  total = len(y_truths)
  correct = 0
  for i in range(len(y_truths)):
    if equal(predicted_values[i],y_truths[i]):
      correct += 1
  acc = correct / total
  return acc


def generate_weights(x_val, y_val, models):
    '''
    returns a list of weights
    '''
    accuracy = []
    weights = np.full((1,len(models)), 100.0)
    for model in models:
      acc = model.evaluate(x_val, y_val)[1]
      accuracy.append(100*acc)
    weights = weights - accuracy
    weights = weights**2
    sum = np.sum(weights)
    weights = weights/sum
    weights = 1/weights
    weights = weights**2
    sum = np.sum(weights)
    weights = weights/sum
    return weights

Writing Ensembling.py


In [10]:
import tensorflow as tf 
import numpy as np
from Ensembling import *
from sklearn.metrics import classification_report, confusion_matrix

x_test_path = input("Enter path to test images: ")
y_test_path = input("Enter path to test labels: ")

inception_path = input("Enter path to Inception Model: ")
resnet_path = input("Enter path to Resnet Model: ")
densenet_path = input("Enter path to DenseNet Model: ")

x_test = np.load(x_test_path)
y_test = np.load(y_test_path)

image_size = 224

inception_model = tf.keras.models.load_model(inception_path)
resnet_model = tf.keras.models.load_model(resnet_path)
densenet_model = tf.keras.models.load_model(densenet_path)

models = [densenet_model,resnet_model,inception_model]

w = generate_weights(x_test,y_test,models)[0] #generating weights
print("Weights: ", w)

predictions = []
for i in range(len(x_test)):
  pred = ensemble(x_test[i].reshape(-1,image_size,image_size,3),w,models)
  predictions.append(pred)

print("Accuracy: ",accuracy(predictions,y_test))

y_pred = np.argmax(np.array(predictions), axis=1)

print("The classification report: ")
print(classification_report(y_pred=y_pred, y_true=y_test))
print()
print("Confusion Matrix: ")
print(confusion_matrix(y_pred=y_pred, y_true=y_test))

Enter path to test images: /content/test/New_Btest_images.npy
Enter path to test labels: /content/test/New_Btest_labels.npy
Enter path to Inception Model: /content/validationinception_v3.h5
Enter path to Resnet Model: /content/validationdensenet201.h5
Enter path to DenseNet Model: /content/validationdensenet201.h5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:4930: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


1/1 [==============================] - 3s 3s/step - loss: 0.0169 - accuracy: 1.0000


/content/Ensembling.py:58: RuntimeWarning: invalid value encountered in true_divide
  weights = weights/sum


Weights:  [nan nan nan]
Accuracy:  0.0
The classification report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00      30.0

    accuracy                           0.00      30.0
   macro avg       0.00      0.00      0.00      30.0
weighted avg       0.00      0.00      0.00      30.0


Confusion Matrix: 
[[ 0  0]
 [30  0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
%%writefile utils.py
import tensorflow as tf 
import numpy as np

categories = ["COVID +ve","COVID -ve"]
image_size = 224

inception_path = './SavedModels/inception_v3.h5'
resnet_path = './SavedModels/resnet50_v2.h5'
densenet_path = './SavedModels/densenet201.h5'
print("Loading Models.....")
inception_model = tf.keras.models.load_model(inception_path)
resnet_model = tf.keras.models.load_model(resnet_path)
densenet_model = tf.keras.models.load_model(densenet_path)
models = [inception_model,resnet_model,densenet_model]
models_name = ["Inception","Resnet","DenseNet"]
print("Models Loaded")

def ensemble(x, weights, models): 
    '''
    returns a weighted average of predictions made by the models\n
    x -> input image \n
    weights -> a list of weights \n
    models -> a list of models\n    
    '''      
    outputs = []    
    for model in models:                
        outputs.append(list(model.predict(x)[0]))                
    
    outputs = np.array(outputs)
    avg = np.average(a=outputs,axis=0,weights=weights)
    return avg

def predict(input_image):
    '''
    returns a predicted class, associated probablity 
    input_image = an image
    models = list of models    
    '''
    weights = [0.172,0.601,0.228]
    input_image = input_image/255.0  
    img = input_image.reshape(-1,image_size,image_size,3)    
    model_predictions = []
    model_probs = []
    for i in range(len(models)):
        print(models_name[i], categories[np.argmax(models[i].predict(img))])
        model_predictions.append(categories[np.argmax(models[i].predict(img))])
        model_probs.append(models[i].predict(img))
    avg_pred = ensemble(img,weights,models)
    if model_predictions.count(categories[0]) == 1:        
        print(categories[0], model_probs[model_predictions.index(categories[0])][0])
    return categories[np.argmax(avg_pred)], avg_pred


Writing utils.py


In [13]:
!pip install matplotlib-venn

In [14]:
!apt-get -qq install -y libfluidsynth1

Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [15]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 160711 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.6_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.6) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.6) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 163kB 7.7MB/s 
  Created wheel for libarchive: filename=libarchive-0.4.7-cp37-none-any.whl size=31633 sha256=fc9f2464c0436b6bac905b4cdae27d74d67313be7648c406e8cf54705b1f4040
  Stored in directory: /root/.cache/pip/wheels/3b/5c/fa/92ee330d259e8fa5bedbd53f67040710fe81cfa463b8711d26
Successfully built libarchive


In [16]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [17]:
!pip install cartopy
import cartopy

     |████████████████████████████████| 12.1MB 246kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 225kB 38.7MB/s 
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516179 sha256=e193117d8eb2799827401a3b67c423c29ff5c1ae943c60c6d8ea698610ae74d1
  Stored in directory: /root/.cache/pip/wheels/45/7f/3b/37879587817fd1bbbee7b47312e2401b4f542cccf2fbe9b4ee
Successfully built cartopy
  Created wheel for pyshp: filename=pyshp-2.1.3-cp37-none-any.whl size=37264 sha256=60b428c9316dbb966c307bdf344bf12aa92b7bafda5c1afb3c66d65023523f6e
  Stored in directory: /root/.cache/pip/wheels/76/2b/d4/53e6b9a0fb0a9f9f29664cf82605af8bc81d5ab44d987896dd
Successfully built pyshp


In [19]:
pip install PyQtWebEngine

     |████████████████████████████████| 235kB 5.3MB/s 
     |████████████████████████████████| 8.3MB 7.1MB/s 
     |████████████████████████████████| 67.5MB 134kB/s 
     |████████████████████████████████| 317kB 31.8MB/s 
     |████████████████████████████████| 59.9MB 54kB/s 


In [20]:
from PyQt5 import QtCore, QtGui, QtWidgets
import sys
from utils import *
import cv2

class Ui_MainWindow(object):
    def setupUi(self, MainWindow):            
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(447, 492)
        MainWindow.setMaximumSize(QtCore.QSize(447, 600))
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.xrayImage = QtWidgets.QLabel(self.centralwidget)
        self.xrayImage.setGeometry(QtCore.QRect(20, 30, 200, 251))
        self.xrayImage.setText("")
        self.xrayImage.setObjectName("xrayImage")
        
        self.predictedLabel = QtWidgets.QLabel(self.centralwidget)
        self.predictedLabel.setGeometry(QtCore.QRect(40, 280, 361, 41))
        font = QtGui.QFont()
        font.setFamily("Segoe UI")
        font.setPointSize(14)
        self.predictedLabel.setFont(font)
        self.predictedLabel.setObjectName("predictedLabel")

        self.browseImageBtn = QtWidgets.QPushButton(self.centralwidget)
        self.browseImageBtn.setGeometry(QtCore.QRect(250, 20, 161, 41))
        font = QtGui.QFont()
        font.setFamily("Segoe UI")
        font.setPointSize(11)
        self.browseImageBtn.setFont(font)
        self.browseImageBtn.setObjectName("browseImageBtn")
        self.browseImageBtn.clicked.connect(self.browseImage)

        self.predictBtn = QtWidgets.QPushButton(self.centralwidget)
        self.predictBtn.setGeometry(QtCore.QRect(250, 80, 161, 41))
        font = QtGui.QFont()
        font.setFamily("Segoe UI")
        font.setPointSize(11)
        self.predictBtn.setFont(font)
        self.predictBtn.setObjectName("predictBtn")
        self.predictBtn.clicked.connect(self.prediction)

        
        self.probLabel = QtWidgets.QLabel(self.centralwidget)
        self.probLabel.setGeometry(QtCore.QRect(40, 320, 111, 41))
        font = QtGui.QFont()
        font.setFamily("Segoe UI")
        font.setPointSize(14)
        self.probLabel.setFont(font)
        self.probLabel.setObjectName("probLabel")

        self.covidPositive_Prob = QtWidgets.QLabel(self.centralwidget)
        self.covidPositive_Prob.setGeometry(QtCore.QRect(40, 360, 361, 41))
        font = QtGui.QFont()
        font.setFamily("Segoe UI")
        font.setPointSize(14)
        self.covidPositive_Prob.setFont(font)
        self.covidPositive_Prob.setObjectName("covidPositive_Prob")
        self.covidPositive_Prob.setWordWrap(True)


        self.covidNegative_Prob = QtWidgets.QLabel(self.centralwidget)
        self.covidNegative_Prob.setGeometry(QtCore.QRect(40, 400, 361, 41))
        font = QtGui.QFont()
        font.setFamily("Segoe UI")
        font.setPointSize(14)
        self.covidNegative_Prob.setFont(font)
        self.covidNegative_Prob.setObjectName("covidNegative_Prob")
        self.covidNegative_Prob.setWordWrap(True)

        MainWindow.setCentralWidget(self.centralwidget)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "COVID-19 Detector"))
        self.predictedLabel.setText(_translate("MainWindow", "PREDICTION:"))
        self.browseImageBtn.setText(_translate("MainWindow", "Browse Image"))
        self.predictBtn.setText(_translate("MainWindow", "Analyse"))
        self.probLabel.setText(_translate("MainWindow", "Probabilities"))
        self.covidPositive_Prob.setText(_translate("MainWindow", "COVID-19 +ve:"))
        self.covidNegative_Prob.setText(_translate("MainWindow", "COVID-19 -ve:"))

    def browseImage(self):
        fm = QtWidgets.QFileDialog.getOpenFileName(None,"OpenFile")
        filename = fm[0]
        self.image = cv2.imread(filename)        
        self.xrayImage.setPixmap(QtGui.QPixmap(filename))
        self.xrayImage.setScaledContents(True)
    
    def prediction(self):
        self.image = cv2.resize(self.image, (image_size,image_size))
        print("Analysis....")        
        try:
            label, probabilities = predict(self.image)   #("COVID",[0.98,0.02])     
            self.predictedLabel.setText("PREDICTION: "+label)
            #print(probabilities)
            self.covidPositive_Prob.setText("COVID-19 +ve: " + str(probabilities[0]))
            self.covidNegative_Prob.setText("COVID-19 -ve: " + str(probabilities[1]))
            print("Analysis done")
        except:
            msgError = QtWidgets.QMessageBox()
            msgError.setIcon(QtWidgets.QMessageBox.Critical)
            msgError.setWindowTitle("Error")
            msgError.setText("Oops!! Error")
            msgError.exec_()
        

if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

Loading Models.....


OSError: ignored